In [1]:
from opensearchpy import OpenSearch
import pandas as pd
import numpy as np
import ast
import json
import os
from dotenv import load_dotenv
from tqdm import tqdm

# Set up connection

In [2]:
load_dotenv(dotenv_path="../server/.env")

OPENSEARCH_URL = os.getenv("OPENSEARCH_URL")
USERNAME = os.getenv("OPENSEARCH_USER")
PASSWORD = os.getenv("OPENSEARCH_PASS")

host = OPENSEARCH_URL.replace("https://", "").replace("http://", "").split(":")[0]
port = 443 if OPENSEARCH_URL.startswith("https") else 9200

client = OpenSearch(
    hosts=[{"host": host, "port": port}],
    http_auth=(USERNAME, PASSWORD),
    use_ssl=OPENSEARCH_URL.startswith("https"),
    verify_certs=False
)

print(f'running on {OPENSEARCH_URL}')

running on https://search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws


c:\Users\nerii\Desktop\Work\DVS\VORD\v1\_deployment\vord\preprocess\env_preprocess\lib\site-packages\opensearchpy\connection\http_urllib3.py:214: UserWarning: Connecting to https://search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


# Clean data

In [3]:
df_dirty = pd.read_csv('data/omniart_CLIP_embeddings.csv')

def strlist_to_str(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    return df

def split_list(df,column):
    df[column] = df[column].apply(lambda x: np.array(x.split(","), dtype=np.float32))
    
    return df

df = strlist_to_str(df_dirty,["color_palette","palette_count"])
df = split_list(df,"CLIP_embeddings")

### Remove empty ids

In [4]:
# Function to convert numpy types to native Python types
def convert_numpy_types(value):
    if isinstance(value, np.integer):  # Handle numpy int types
        return int(value)
    elif isinstance(value, np.floating):  # Handle numpy float types
        return float(value)
    elif isinstance(value, np.ndarray):  # Handle numpy arrays
        return value.tolist()
    else:
        return value

empty = [42205, 37898, 35589, 42206, 43035, 25831, 5018206, 38947, 18792, 5030227, 25566, 5018053, 24354, 9206, 23536, 17400, 40940, 5194, 5025499, 12679, 32150, 5025337, 35381, 5018117, 6403, 17173, 34261, 5022273, 25768, 5842, 40839, 5237, 26246, 3471, 5021478, 5021511, 39410, 28448, 38918, 42225, 4639, 22680, 10660, 5030337, 5030237, 20143, 24070, 19177, 38033, 39523, 5030329, 5028755, 6830, 15510, 26308, 7555, 33877, 35101, 30005, 23593, 34568, 3258, 5011, 40879, 19246, 6912, 2734, 24220, 11401, 24393, 28099, 5025837, 2000199, 3409, 6607, 41158, 27046, 24718, 2587, 30638, 9368, 27047, 36555, 3039, 340, 5024847, 4216, 39490, 15211, 17835, 24340, 23216, 40835, 40270, 3263, 16909, 16923, 2000050, 9457, 42895, 15406, 24137, 21134, 11910, 32089, 24810, 27845, 11888, 6947, 31417, 606, 24591, 5028251, 42372, 38823, 5019172, 5165922, 29948, 5030191, 5030498, 5028589, 37874, 5025826, 24294, 36743, 3586, 39638, 34951, 5344, 5030525, 37393, 12723, 30091, 5018052, 37408, 39165, 17481, 6404, 5028321, 5571, 6938, 327, 3119, 16865, 41742, 3410, 35906, 11341, 7001, 14046, 39173, 13124, 26915, 3331, 35576, 17352, 22825, 5030474, 32225, 5025861, 13686, 34237, 18079, 7245, 5021104, 20528, 14792, 37039, 5024875, 43479, 9337, 16609, 22908, 5018110, 36656, 8218, 29158, 8527, 11912, 22749, 36623, 18741, 5072292, 13479, 6405, 7151, 27638, 26166, 4630, 5058073, 5030445, 32547, 5021106, 39854, 39105, 3643, 1132, 25661, 5022264, 44106, 36904, 34213, 34285, 18494, 42377, 13243, 21939, 5022256, 42269, 2768, 17890, 39887, 29073, 43727, 9221, 20188, 33507, 7522, 15312, 43028, 39135, 5033079, 5025843, 19538, 5030230, 5034700, 38886, 6879, 13724, 22713, 27099, 29227, 5028578, 22313, 22045, 42879, 3136, 25704, 32815]

for i in empty:
    df = df[df['omni_id'] != i]

# Convert the entire DataFrame to native Python types
df_cleaned = df.applymap(convert_numpy_types)

# Use the DataFrame index as the '_id'
df_cleaned['_id'] = df_cleaned["id"]

C:\Users\nerii\AppData\Local\Temp\ipykernel_19520\4057418748.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(convert_numpy_types)


### Reduce columns based on schema and check type

In [ ]:
schema = pd.read_json('data/schema.json')
columns = schema.paintings.mappings["properties"].keys()
schema_dtypes = {col: schema.paintings.mappings["properties"][col]["type"] for col in columns}

# Filter df based on schema
df_reduced = df_cleaned[list(schema_dtypes.keys())]

# Map the schema data types to pandas data types
dtype_mapping = {
    "text": "str",
    "keyword": "str",
    "float": "float64",
    "integer": "int32",
    "date": "int32",
    "knn_vector": "object"
}

dtype_lists = ["color_palette", "palette_count", "CLIP_embeddings"]

# Convert the data types of df_reduced to match the schema
for col, dtype in schema_dtypes.items():
    if col in dtype_lists:
        continue
    if dtype in dtype_mapping:
        try:
            df_reduced[col] = df_reduced[col].astype(dtype_mapping[dtype])
        except:
            print(f"Could not convert {col} to {dtype}")

# df_reduced['creation_year'] = df_reduced['creation_year'].astype(str).str.zfill(4)

C:\Users\nerii\AppData\Local\Temp\ipykernel_19520\627449997.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced[col] = df_reduced[col].astype(dtype_mapping[dtype])
C:\Users\nerii\AppData\Local\Temp\ipykernel_19520\627449997.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced[col] = df_reduced[col].astype(dtype_mapping[dtype])
C:\Users\nerii\AppData\Local\Temp\ipykernel_19520\627449997.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

# Export data

In [32]:
# Convert Dataframe to JSON
df_reduced.to_json('outputs/df_paintings.jsonl', orient='records', lines=True)

In [30]:
df_reduced["creation_year"].to_list()

['1620',
 '1619',
 '1465',
 '1277',
 '1433',
 '1666',
 '1559',
 '1725',
 '1625',
 '1725',
 '1526',
 '1526',
 '1506',
 '1649',
 '1484',
 '1849',
 '1815',
 '1430',
 '1553',
 '1606',
 '1295',
 '1675',
 '1475',
 '1308',
 '1500',
 '1477',
 '1655',
 '1659',
 '1640',
 '1867',
 '1470',
 '1659',
 '1705',
 '1435',
 '1473',
 '1642',
 '1497',
 '1685',
 '1675',
 '1695',
 '1773',
 '1510',
 '1625',
 '1283',
 '1730',
 '1685',
 '1654',
 '1488',
 '1476',
 '1725',
 '1700',
 '1612',
 '1625',
 '1526',
 '1620',
 '1447',
 '1395',
 '1651',
 '1775',
 '1725',
 '1625',
 '1180',
 '1540',
 '1459',
 '1560',
 '1809',
 '1552',
 '1622',
 '1632',
 '1888',
 '1554',
 '1599',
 '1702',
 '1675',
 '1502',
 '1435',
 '1705',
 '1500',
 '1553',
 '1308',
 '1873',
 '1675',
 '1570',
 '1625',
 '1625',
 '1555',
 '1540',
 '1308',
 '1805',
 '1675',
 '1775',
 '1885',
 '1732',
 '1504',
 '1755',
 '1899',
 '1656',
 '1863',
 '1824',
 '1770',
 '1580',
 '1800',
 '1625',
 '1404',
 '1050',
 '1325',
 '1604',
 '1869',
 '1550',
 '1510',
 '1452',
 

# Upload data

In [ ]:
INDEX_NAME = "paintings"  

# Convert DataFrame to a list of dictionaries
# df_dict = df_reduced.to_dict(orient='records')
df_dict = df_reduced.to_dict(orient='records')

### Test the first entry

In [9]:
# Upload test
response = client.index(index=INDEX_NAME, body=df_dict[0])
doc_id = response['_id']
print(f"✅ Indexed painting: {doc_id}")

c:\Users\nerii\Desktop\Work\DVS\VORD\v1\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


✅ Indexed painting: r1Qo05UBRvMkBlsWDMpO


In [10]:
# Delete test
delete_response = client.delete(index=INDEX_NAME, id=doc_id)
print(f"🗑️ Deleted painting: {delete_response['result']}")

c:\Users\nerii\Desktop\Work\DVS\VORD\v1\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


🗑️ Deleted painting: deleted


### Iterate on all entries

In [11]:
# Index each painting
failed = []

for painting in tqdm(df_dict, desc="Indexing paintings", unit="painting"):  
    try:
        response = client.index(index=INDEX_NAME, body=painting)
    except Exception as e:
        failed.append(painting)
        # print(f"❌ Error indexing painting: {e}")

Indexing paintings:   0%|          | 0/2761 [00:00<?, ?painting/s]c:\Users\nerii\Desktop\Work\DVS\VORD\v1\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\nerii\Desktop\Work\DVS\VORD\v1\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Indexing paintings:   0%|          | 2/2761 [00:00<02:40, 17.19painting/s]c:\Users\nerii\Desktop\Work\DVS\

In [12]:
len(failed)

17

# Check Upload

In [4]:
INDEX_NAME = "paintings"  

response = client.search(
    index=INDEX_NAME,
    body={
        "query": { "match_all": {} },
        "size": 10  # limit number of docs returned
    }
)

for hit in response["hits"]["hits"]:
    print(json.dumps(hit["_source"], indent=2))

c:\Users\nerii\Desktop\Work\DVS\VORD\v1\_deployment\vord\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{
  "id": "dd5bafd3-0acb-44ba-b2e5-c05c998937b0",
  "artwork_name": "st sebastian",
  "artist_full_name": "nicolas rgnier",
  "artist_first_name": "nicolas",
  "artist_last_name": "rgnier",
  "creation_year": "1620",
  "century": 17,
  "source_url": "https://www.wga.hu/html/r/regnier/sebastia.html",
  "image_url": "http://www.wga.hu/art/r/regnier/sebastia.jpg",
  "collection_origins": "Web Gallery of Art '17",
  "artwork_type": "painting",
  "school": "french",
  "original_id_in_collection": 32925,
  "omni_id": 32918,
  "general_type": "painting",
  "color_palette": [
    "#b58125",
    "#26170b",
    "#221107",
    "#21150d",
    "#1b0f09",
    "#130d09",
    "#d9a44c",
    "#795010",
    "#2e1a0a",
    "#1d1008",
    "#180e08",
    "#140a05"
  ],
  "dominant_color": "#21150d",
  "palette_count": [
    7321,
    4736,
    6194,
    7405,
    5969,
    4837,
    3675,
    4747,
    5387,
    6034,
    4831,
    4400
  ],
  "CLIP_embeddings": [
    0.0132064819,
    0.031829834,
    -0.

In [5]:
INDEX_NAME = "paintings"

def count_documents(client, index_name):
    response = client.count(
        index=index_name,
        body={
            "query": {
                "match_all": {}
            }
        }
    )
    return response['count']

total_docs = count_documents(client, INDEX_NAME)
print(f"Total documents in index '{INDEX_NAME}': {total_docs}")

c:\Users\nerii\Desktop\Work\DVS\VORD\v1\_deployment\vord\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Total documents in index 'paintings': 2761
